# Procedure Documentation
* In this notebook, we provide all logs in shortened form (omit DEBUG logs) and go over each procedure and decisions following it.
* To repeat, a "procedure" in this project is just a structured way of executing "tasks", where as a task is defined the translation of a set of pairs $P$ using sentences from dataset $d$ with the translator $t$, the only thing a procedure does is allowing us to run multiple tasks by specifiying $t$ and $d$ using CLI commands, whereas $P$ is fixed in the respective `proc.py` script.

## Procedure 1
* Translated 20 pairs, from and into English (i.e. en-de, de-en, etc.) from datasets OPUS100, EuroParl and FLORES+ with both GPT4.1 and DeepL.
* It acted as an 'initial' test to see if everything worked as expected or unaccounted issues occur that may require adjustment for subsequent procedure.
* As we see in the logs, no big issues occured. 
* There were automatic retries triggered by OpenAI's implementation, not the implementation by us, but at that time, they were not too noticable/worrisome, making us believe we could start the next procedures without adjustments.
* `proc1.py` was thus executed fully with the following commands:
```sh
python proc1.py run -m gpt-4.1-2025-04-14 -d europarl
python proc1.py run -m deepl_document -d europarl
python proc1.py run -m gpt-4.1-2025-04-14 -d opus-100
python proc1.py run -m deepl_document -d opus-100
python proc1.py run -m gpt-4.1-2025-04-14 -d flores_plus
python proc1.py run -m deepl_document -d flores_plu
```
* 20 pairs, 400 sentences, 3 datasets, 2 translators amounting to 6 tasks, thus 6 CLI commands.

In [1]:
# Collect 6 task ids, as proc1 consisted of 6 tasks
from os.path import join
import os
import json
task_folder = 'tasks'
proc1 = join(task_folder, 'proc1')
proc1_ep = join(proc1, 'europarl')
proc1_opus = join(proc1, 'opus-100')
proc1_flores = join(proc1, 'flores_plus')
translators = ['deepl_document', 'gpt-4.1-2025-04-14']
task_folders = [proc1_ep, proc1_opus, proc1_flores]
proc1_task_ids = []

for tl in translators:
    for folder in task_folders:
        with open(join(folder, tl, 'task.json'), 'r') as f:
            task = json.load(f)
            proc1_task_ids.append(task['task_id'])
len(proc1_task_ids)

6

In [2]:
# Collect structured logs stored on proc1.jsonl
with open(join(task_folder, 'proc1.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]

proc1_task_id2logs = {task_id: [] for task_id in proc1_task_ids}
for task_id in proc1_task_ids:
    for log in logs:
        if log['id'].startswith(task_id):
            proc1_task_id2logs[task_id].append(log)

In [3]:
for task_id, logs in proc1_task_id2logs.items():
    print(task_id, len(logs))

ab9fc42b-61af-4534-afd6-3710eb1dbc52 20
4c5ffc31-c043-414d-aba1-270f9967f078 20
88538d1d-11fc-4356-9153-b105d4e26e30 20
9549e927-4f63-4205-82bb-5c7ccabfe943 20
fa628bc1-4f85-446d-9167-1a8d99ccc493 20
fdbf6190-d061-4154-ae94-d7b11199d043 20


* We see 20 logs for each task as expected

In [4]:
for tl in translators:
    for folder in task_folders:
        files = os.listdir(join(folder, tl))
        print(folder, tl, len(files))

tasks\proc1\europarl deepl_document 21
tasks\proc1\opus-100 deepl_document 21
tasks\proc1\flores_plus deepl_document 21
tasks\proc1\europarl gpt-4.1-2025-04-14 21
tasks\proc1\opus-100 gpt-4.1-2025-04-14 21
tasks\proc1\flores_plus gpt-4.1-2025-04-14 21


* We find 21 files per task, 20 txt files for 20 translations and `task.json` that contains some meta information the task itself.

In [5]:
print('Proc1', f'{sum([len(logs) for logs in proc1_task_id2logs.values()])}/120')

Proc1 120/120


In [6]:
!cat proc1.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-07 13:55:45 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-07 13:55:45 - [🏁]: Starting task 9549e927-4f63-4205-82bb-5c7ccabfe943 on commit 66ef922
INFO: 2025-05-07 13:59:58 - [✔️]: Translated 398 sents for de-en
INFO: 2025-05-07 14:03:12 - [✔️]: Translated 400 sents for en-de
INFO: 2025-05-07 14:05:34 - [✔️]: Translated 400 sents for da-en
INFO: 2025-05-07 14:08:09 - [✔️]: Translated 400 sents for en-da
INFO: 2025-05-07 14:12:09 - [✔️]: Translated 400 sents for el-en
INFO: 2025-05-07 14:16:08 - [✔️]: Translated 400 sents for en-el
INFO: 2025-05-07 14:17:37 - [✔️]: Translated 400 sents for pt-en
INFO: 2025-05-07 14:19:38 - [✔️]: Translated 400 sents for en-pt
INFO: 2025-05-07 14:21:29 - [✔️]: Translated 400 sents for sv-en
INFO: 2025-05-07 14:23:14 - [✔️]: Translated 400 sents for en-sv
INFO: 2025-05-07 14:25:11 - [✔️]: Translated 401 sents for es-en
INFO: 2025-05-07 14:27:50 - [✔️]: Translated 400 sents for en-es
INFO: 2025-05-07 14:30:10 - [✔️]: Tra

### Observations
* We do not include all logs, as some of them are not that informative. 
    * We include the logs created by us (indicated with emoji within `[]`)
    * We include the `Retry` log implemented by OpenAI developers of the OpenAI Python package, as that one became relevant for the next procedures.
* Overall, procedure 1 completed with many issues, there were only two retries, in task europarl-gpt and task flores-gpt.
* We make an observation that it takes substantially longer for GPT4.1 to translate than DeepL.

In [7]:
!cat proc1.log | grep -P "Retrying request to /chat/"

INFO: 2025-05-07 14:35:11 - Retrying request to /chat/completions in 0.437165 seconds
INFO: 2025-05-07 16:00:09 - Retrying request to /chat/completions in 0.422277 seconds


In [8]:
!cat proc1.log | grep -P "(Selected task|Task took)"

INFO: 2025-05-07 13:55:45 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-07 14:57:16 - [🏁]: Task took 3690.83s
INFO: 2025-05-07 14:57:36 - [🚀]: Selected task for europarl - deepl_document
INFO: 2025-05-07 15:01:05 - [🏁]: Task took 208.54s
INFO: 2025-05-07 15:01:52 - [🚀]: Selected task for opus-100 - gpt-4.1-2025-04-14
INFO: 2025-05-07 15:38:20 - [🏁]: Task took 2187.31s
INFO: 2025-05-07 15:38:48 - [🚀]: Selected task for opus-100 - deepl_document
INFO: 2025-05-07 15:41:18 - [🏁]: Task took 149.83s
INFO: 2025-05-07 15:41:35 - [🚀]: Selected task for flores_plus - gpt-4.1-2025-04-14
INFO: 2025-05-07 16:39:43 - [🏁]: Task took 3487.97s
INFO: 2025-05-07 16:39:58 - [🚀]: Selected task for flores_plus - deepl_document
INFO: 2025-05-07 16:42:50 - [🏁]: Task took 172.57s


## Procedure 2

* In procedure 2, we wanted to translate the remaining 90 pairs (so all pairs that did not include `en` as source or target language) of EuroParl and FLORES+ using GPT4.1
* We decided to split it into 45 pairs, estimating a translation time of roughly more than 2 hours, as in procedure 1, it took 1 hour for EuroParl and FLORES+. 
    * It took half as much for OPUS100 because OPUS100 has shorter sentences on average (less characters, less tokens)
* The 45 pair batches were not random but had order, to replicate them in case of errors. 
* Each batch of 45 pairs could be segmented into sub-batches of 9 pairs, where the source language was fixed (de-da, de-fi, de-el, etc.)
* The plan was thus to run 4 tasks, 4 CLI commands but we decided to interrupt procedure 2 after the first task has been completed, thus only the following command was run:
```sh
python proc2.py run -m gpt-4.1-2025-04-14 -d europarl-1
```
* The `europarl-1` indicates the first batch of 45 pairs from the europarl dataset. It also shows a slight flaw in design philosophy of this procedure based approach of running tasks: The set of pairs $P$ is fixed on script level, it is not possible to specify them within the CLI command. Works out if $|P|$ is large, becomes problematic if small.
* Translated 44/45 pairs (source languages: da, de, el, es, fi) from the EuroParl dataset, failed for the pair fi-el due a Gateway Timeouts exceeding not only OpenAI's automatic retries but also the automatic retries implemented by us.

In [9]:
proc2 = join(task_folder, 'proc2')
proc2_task_file = join(proc2, join('europarl', join('gpt-4.1-2025-04-14', 'task.json')))
with open(proc2_task_file, 'r') as f:
    proc2_task = json.load(f)
    proc2_task_id = proc2_task['task_id']

with open(join(task_folder, 'proc2.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]

proc2_logs = [log for log in logs if log['id'].startswith(proc2_task_id)]
print(proc2_task_id, len(proc2_logs))
# Failed fi-el

cfaadc55-8ced-4b93-80e6-9581c956f986 44


* Only 44 logs, as we decided to not create logs in cases where we did not receive a response from the API, i.e. errors.

In [10]:
files = os.listdir(join(proc2, join('europarl', 'gpt-4.1-2025-04-14')))
print(join(proc2, join('europarl', 'gpt-4.1-2025-04-14')), len(files))

tasks\proc2\europarl\gpt-4.1-2025-04-14 45


* 45 logs, again 44 translations and the `task.json` file.

In [11]:
print('Proc2', f'{len(proc2_logs)}/45')

Proc2 44/45


In [12]:
!cat proc1.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-07 13:55:45 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-07 13:55:45 - [🏁]: Starting task 9549e927-4f63-4205-82bb-5c7ccabfe943 on commit 66ef922
INFO: 2025-05-07 13:59:58 - [✔️]: Translated 398 sents for de-en
INFO: 2025-05-07 14:03:12 - [✔️]: Translated 400 sents for en-de
INFO: 2025-05-07 14:05:34 - [✔️]: Translated 400 sents for da-en
INFO: 2025-05-07 14:08:09 - [✔️]: Translated 400 sents for en-da
INFO: 2025-05-07 14:12:09 - [✔️]: Translated 400 sents for el-en
INFO: 2025-05-07 14:16:08 - [✔️]: Translated 400 sents for en-el
INFO: 2025-05-07 14:17:37 - [✔️]: Translated 400 sents for pt-en
INFO: 2025-05-07 14:19:38 - [✔️]: Translated 400 sents for en-pt
INFO: 2025-05-07 14:21:29 - [✔️]: Translated 400 sents for sv-en
INFO: 2025-05-07 14:23:14 - [✔️]: Translated 400 sents for en-sv
INFO: 2025-05-07 14:25:11 - [✔️]: Translated 401 sents for es-en
INFO: 2025-05-07 14:27:50 - [✔️]: Translated 400 sents for en-es
INFO: 2025-05-07 14:30:10 - [✔️]: Tra

### Observations
* We observe that automatic retries were triggered much more frequently, in fact, not even just by OpenAI but also ours. 
* OpenAI's automatic retries employ something along the lines of an exponential backoff, it retries after roughly 0.4s and then again after roughly 0.8s
* Our own automatic retries are indicated with the 🕒 emoji and call the API again after exactly 30 seconds.
* These automatic retries caused an extremely long waiting time for a single task to finish; interfering with the author's student schedule, as they were still monitoring the task progress and had to remain connected to the Internet. 
    * In case of Internet failures, the 30 seconds timeframe of our automatic retry gives room to connect back and continue the task, avoiding error propagations.

In [13]:
!cat proc2.log | grep -P "(Retry|skipping)"

DEBUG: 2025-05-08 11:14:57 - Retrying due to status code 504
INFO: 2025-05-08 11:14:57 - Retrying request to /chat/completions in 0.427331 seconds
DEBUG: 2025-05-08 11:45:25 - Retrying due to status code 504
INFO: 2025-05-08 11:45:25 - Retrying request to /chat/completions in 0.390387 seconds
DEBUG: 2025-05-08 11:50:26 - Retrying due to status code 504
INFO: 2025-05-08 11:50:26 - Retrying request to /chat/completions in 0.875335 seconds
INFO: 2025-05-08 11:55:27 - [🕒]: Retrying de-el...
DEBUG: 2025-05-08 12:00:58 - Retrying due to status code 504
INFO: 2025-05-08 12:00:58 - Retrying request to /chat/completions in 0.484229 seconds
DEBUG: 2025-05-08 12:28:38 - Retrying due to status code 502
INFO: 2025-05-08 12:28:38 - Retrying request to /chat/completions in 0.387740 seconds
DEBUG: 2025-05-08 12:31:15 - Retrying due to status code 502
INFO: 2025-05-08 12:31:15 - Retrying request to /chat/completions in 0.834205 seconds
DEBUG: 2025-05-08 13:54:27 - Retrying due to status code 504
INFO: 

* Most retries were caused by Gateway Timeouts indicated by the status code 504, for a certain instance it was 502.
* We checked the OpenAI developer forum for this but most suggestions at that time just informed us to either reduce the prompt size or employ automatic retries with delays, which we have already done.
    * [Forum Thread1](https://community.openai.com/t/api-error-504-in-call-to-chatopenai/292512/3): Mentions prompt size, rate limits, internet connection or checking OpenAI status.
    * [Forum Thread2](https://community.openai.com/t/504-gateway-time-out-error/720369): Mentions reducing prompt size
* Since we were able to get 44 pairs with 400 sentences and we knew it could not be the rate limits, as the logs did not indicate such, we decided to translate the remaining pairs of interupted procedure 2 in procedure 3 by reducing the size of each task even further.

In [14]:
!cat proc2.log | grep -P "(Selected task|Task took|skipping)"

INFO: 2025-05-08 10:59:02 - [🚀]: Selected task for europarl-1 - gpt-4.1-2025-04-14
INFO: 2025-05-08 14:40:10 - [⏭️]: Failed 2 times, skipping fi-el...
INFO: 2025-05-08 15:04:01 - [🏁]: Task took 14699.12s


* We planned roughly more than 2 hours for one task but it took slightly more than 4 hours.

## Procedure 3

* Continues where procedure 2 was interrupted. Instead of translating 45 pairs, we translated now the segments within those 45, i.e. 9 pairs with source language fixed (de-da, de-fi, de-el, etc.) 
* This results in 15 tasks, 5 tasks for the remaining EuroParl pairs and 10 tasks for the FLORES+ pairs.
* However, again, we decided to interrupt procedure 3 after 5 tasks of the EuroParl pairs have been completed, thus, only the following CLI commands were ran:
```sh
python proc3.py run -m gpt-4.1-2025-04-14 -d europarl-fr
python proc3.py run -m gpt-4.1-2025-04-14 -d europarl-it
python proc3.py run -m gpt-4.1-2025-04-14 -d europarl-nl
python proc3.py run -m gpt-4.1-2025-04-14 -d europarl-pt
python proc3.py run -m gpt-4.1-2025-04-14 -d europarl-sv
```
* This resulted in 42/45 translated pairs, it-el, it-fi and sv-el were skipped due to frequent Gateway Timeouts
* Furthermore, we noticed that OpenAI provides a [logging feature for Chat Completion on the OpenAI platform](https://platform.openai.com/logs?api=chat-completions). 
* It was disabled by default. We decided to enable it for procedure 3 out of curiosity (Spoiler: this turned out to be a significant decision). 

In [15]:
proc3 = join(task_folder, 'proc3')
proc3_folders = os.listdir(proc3)
proc3_task_ids = []
for folder in proc3_folders:
    if folder.startswith('flores'): continue
    with open(join(proc3, join(folder, join('gpt-4.1-2025-04-14', 'task.json'))), 'r') as f:
        task = json.load(f)
        proc3_task_ids.append(task['task_id'])

with open(join(task_folder, 'proc3.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]

proc3_task_id2logs = {task_id: [] for task_id in proc3_task_ids}
for task_id in proc3_task_ids:
    for log in logs:
        if log['id'].startswith(task_id):
            proc3_task_id2logs[task_id].append(log)

total = 0
for task_id, logs in proc3_task_id2logs.items():
    total += len(logs)
    print(task_id, len(logs))

4556b5df-51b7-4d7d-a545-e53943fa64a3 9
1077cd91-85d1-4100-a038-acdba34d7e6c 7
7de67252-5e5d-4cfe-8f6c-3a9d837389a0 9
71ac2b3d-eff1-4291-9a62-8f94ac0ac770 9
0c97e7cc-220d-4aae-93d2-c3ca4884ef75 8


* 9 logs per task, 7 for `europarl-it` as it-fi and it-el were skipped and 8 for `europarl-sv`, as sv-el was skipped.

In [16]:
for task in [t for t in os.listdir(proc3) if t.startswith('europarl')]:
    files = os.listdir(join(proc3, join(task, 'gpt-4.1-2025-04-14')))
    print(join(proc3, join(task, 'gpt-4.1-2025-04-14')), len(files))

tasks\proc3\europarl-fr\gpt-4.1-2025-04-14 10
tasks\proc3\europarl-it\gpt-4.1-2025-04-14 8
tasks\proc3\europarl-nl\gpt-4.1-2025-04-14 10
tasks\proc3\europarl-pt\gpt-4.1-2025-04-14 10
tasks\proc3\europarl-sv\gpt-4.1-2025-04-14 9


* Same numbers again, including `task.json`

In [17]:
print('Proc3', f'{total}/45')

Proc3 42/45


In [18]:
!cat proc1.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-07 13:55:45 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-07 13:55:45 - [🏁]: Starting task 9549e927-4f63-4205-82bb-5c7ccabfe943 on commit 66ef922
INFO: 2025-05-07 13:59:58 - [✔️]: Translated 398 sents for de-en
INFO: 2025-05-07 14:03:12 - [✔️]: Translated 400 sents for en-de
INFO: 2025-05-07 14:05:34 - [✔️]: Translated 400 sents for da-en
INFO: 2025-05-07 14:08:09 - [✔️]: Translated 400 sents for en-da
INFO: 2025-05-07 14:12:09 - [✔️]: Translated 400 sents for el-en
INFO: 2025-05-07 14:16:08 - [✔️]: Translated 400 sents for en-el
INFO: 2025-05-07 14:17:37 - [✔️]: Translated 400 sents for pt-en
INFO: 2025-05-07 14:19:38 - [✔️]: Translated 400 sents for en-pt
INFO: 2025-05-07 14:21:29 - [✔️]: Translated 400 sents for sv-en
INFO: 2025-05-07 14:23:14 - [✔️]: Translated 400 sents for en-sv
INFO: 2025-05-07 14:25:11 - [✔️]: Translated 401 sents for es-en
INFO: 2025-05-07 14:27:50 - [✔️]: Translated 400 sents for en-es
INFO: 2025-05-07 14:30:10 - [✔️]: Tra

In [19]:
!cat proc3.log | grep -P "(Retry|skipping|Translated)"

INFO: 2025-05-09 13:22:10 - [✔️]: Translated 400 sents for fr-da
INFO: 2025-05-09 13:26:42 - [✔️]: Translated 400 sents for fr-de
DEBUG: 2025-05-09 13:31:42 - Retrying due to status code 504
INFO: 2025-05-09 13:31:42 - Retrying request to /chat/completions in 0.417134 seconds
INFO: 2025-05-09 13:36:34 - [✔️]: Translated 400 sents for fr-el
INFO: 2025-05-09 13:39:17 - [✔️]: Translated 400 sents for fr-es
DEBUG: 2025-05-09 13:49:06 - Retrying due to status code 504
INFO: 2025-05-09 13:49:06 - Retrying request to /chat/completions in 0.460105 seconds
DEBUG: 2025-05-09 13:54:07 - Retrying due to status code 504
INFO: 2025-05-09 13:54:07 - Retrying request to /chat/completions in 0.971100 seconds
INFO: 2025-05-09 13:58:41 - [✔️]: Translated 400 sents for fr-fi
INFO: 2025-05-09 14:02:23 - [✔️]: Translated 400 sents for fr-it
INFO: 2025-05-09 14:07:12 - [✔️]: Translated 400 sents for fr-nl
INFO: 2025-05-09 14:10:56 - [✔️]: Translated 400 sents for fr-pt
INFO: 2025-05-09 14:15:46 - [✔️]: Trans

In [20]:
!cat proc3.log | grep -P "(Selected task|Task took|skipping)"

INFO: 2025-05-09 13:18:06 - [🚀]: Selected task for europarl-fr - gpt-4.1-2025-04-14
INFO: 2025-05-09 14:15:46 - [🏁]: Task took 3460.14s
INFO: 2025-05-09 14:17:04 - [🚀]: Selected task for europarl-it - gpt-4.1-2025-04-14
INFO: 2025-05-09 15:20:54 - [⏭️]: Failed 2 times, skipping it-el...
INFO: 2025-05-09 16:10:04 - [⏭️]: Failed 2 times, skipping it-fi...
INFO: 2025-05-09 16:26:07 - [🏁]: Task took 7742.83s
INFO: 2025-05-09 16:28:03 - [🚀]: Selected task for europarl-nl - gpt-4.1-2025-04-14
INFO: 2025-05-09 17:48:07 - [🏁]: Task took 4803.96s
INFO: 2025-05-09 19:19:38 - [🚀]: Selected task for europarl-pt - gpt-4.1-2025-04-14
INFO: 2025-05-09 20:26:53 - [🏁]: Task took 4035.21s
INFO: 2025-05-09 20:27:10 - [🚀]: Selected task for europarl-sv - gpt-4.1-2025-04-14
INFO: 2025-05-09 21:21:34 - [⏭️]: Failed 2 times, skipping sv-el...
INFO: 2025-05-09 21:41:08 - [🏁]: Task took 4438.69s


### Observation
* We observe that this frequent occurance of the Gateway Timeout really made the translation task last substantially longer than it should. * Additionally, we should have disabled OpenAI's automatic retries. we checked the logs on OpenAI's platform and observed that each automatic retry triggered by OpenAI actually generated a successful response, meaning, we were getting charged for responses that we were not receiving directly by the API.
    * **NOTE**: In hindsight, we found out after the complection of all procedures, that it was possible to still get these responses through code using API calls. This opens the possibility of investigating determinism as we have now duplicate outputs that could be compared, generated using the exact same input prompts and hyper parameters. However, we decided to not pursue this as it would exceed the scope of the thesis too much. **All translations in this project are associated with logs that are documented.**
* We consulted the OpenAI Developer forum once again and learned that we could perhaps avoid this issue by using streaming instead of request-responses. We would only get charged for responses we actually receive and automatic retries were set to 0 by default for streaming, so only our own retries would be triggered. 
    * [Forum Thread](https://community.openai.com/t/504-gateway-timeout-but-response-logged/1256333) 
* Quick experiments with streaming revealed that it seemed somewhat faster as well, it did not fully remove the occurence of errors but felt more stable, making us decide to use streaming for the last 10 batches of 9 pairs for FLORES+ in procedure 4.


## Procedure 4

* Continues where procedure 2 and 3 left off, translating 90 pairs, in batches of 9 pairs with fixed source language (de-da, de-fi, de-el, etc.) of the FLORES+ dataset.
* This amounts to exactly 10 tasks which were executed with the following CLI commands:
```sh
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-da
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-de
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-el
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-es
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-fi
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-fr
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-it
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-nl
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-pt
python proc4.py run -m gpt-4.1-2025-04-14 -d flores_plus-sv
```
* 89/90 pairs were translated, only skipping de-fi due to failing the acceptance condition. 
    * The number of output lines was larger than the max we provided, we accpeted output between 360-480 lines or sentences. 
    * 360 being the minimum we were willing to accept, giving it 60 sentences leeway before going below 300. 
    * 480 sentences were based on the maximum number of sentences we could detect in the datasets, if we join the sentences and split them again using [SentenceSplitter](https://github.com/mediacloud/sentence-splitter)
    * In this case, the pair for `de-fi` had roughly 1200 sentences in the translation because GPT4.1 decides to return it in a format, where each German sentence was included with translated Finnish sentence, despite the prompt asking it to only provide the translation. 

In [21]:
proc4 = join(task_folder, 'proc4')
proc4_folders = os.listdir(proc4)
proc4_task_ids = []
for folder in proc4_folders:
    with open(join(proc4, join(folder, join('gpt-4.1-2025-04-14', 'task.json'))), 'r') as f:
        task = json.load(f)
        proc4_task_ids.append(task['task_id'])

with open(join(task_folder, 'proc4.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]

proc4_task_id2logs = {task_id: [] for task_id in proc4_task_ids}
for task_id in proc4_task_ids:
    for log in logs:
        if log['id'].startswith(task_id) and log['verdict'] == 'accepted':
            proc4_task_id2logs[task_id].append(log)
        if log['id'].startswith(task_id) and log['verdict'] == 'rejected':
            print('failure', log['src_lang'], log['tgt_lang'], log['id'])
            print('\tfailure stats:')
            print(f'\tout_lines: {log['out_lines']}')
            print(f'\tout_sents: {log['out_sents']}')

total = 0
for task_id, logs in proc4_task_id2logs.items():
    total += len(logs)
    print(task_id, len(logs))

failure de fi ee3252e9-ee96-4559-854b-f2ac3d0e912c-0005
	failure stats:
	out_lines: 1199
	out_sents: 1288
1a161576-a298-4ff2-ba5c-7e321d983122 9
ee3252e9-ee96-4559-854b-f2ac3d0e912c 8
4c842a2a-3e69-43d6-86e1-db7207707967 9
0505b4c1-a077-4577-8560-8620f87f7cfb 9
3f2602f2-ece0-4c2f-9845-e1dc3d00a96a 9
80796dbf-4543-4bc5-ade7-5b15fefb791f 9
fb4d9a89-831d-4d6a-8257-b3a39434f8a6 9
42fb35e2-4bd2-4a46-a517-8b738bae923e 9
a67edffb-c282-4ede-827e-15508a20850a 9
e1e047f6-cf94-423c-b89b-2dab39c05fd7 9


* We have 9 logs per task because failure of acceptance conditions are logged, as we still got an output back.

In [22]:
for task in [t for t in os.listdir(proc4)]:
    files = os.listdir(join(proc4, join(task, 'gpt-4.1-2025-04-14')))
    print(join(proc4, join(task, 'gpt-4.1-2025-04-14')), len(files))

tasks\proc4\flores_plus-da\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-de\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-el\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-es\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-fi\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-fr\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-it\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-nl\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-pt\gpt-4.1-2025-04-14 10
tasks\proc4\flores_plus-sv\gpt-4.1-2025-04-14 10


In [23]:
!ls tasks/proc4/flores_plus-de/gpt-4.1-2025-04-14/

de-da.txt
de-el.txt
de-es.txt
de-fi_fail2.txt
de-fr.txt
de-it.txt
de-nl.txt
de-pt.txt
de-sv.txt
task.json


* Same as in the previous cases, including `task.json`, the file for `de-fi` is called `de-fi-fail2.txt`
    * `fail2` because of an issue in the task code we decided to not fix for the remaining procedures either. If an error occurs before failure, triggering an automatic retry of our code, then the number of retries is incremented and the 2 indicates that it was the second retry that resulted in a failure, the first one being an error.

In [24]:
print('Proc4', f'{total}/90')

Proc4 89/90


In [25]:
!cat proc4.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-10 10:43:59 - [🚀]: Selected task for flores_plus-da - gpt-4.1-2025-04-14
INFO: 2025-05-10 10:43:59 - [🏁]: Starting task 1a161576-a298-4ff2-ba5c-7e321d983122 on commit 7124518
INFO: 2025-05-10 10:47:00 - [✔️]: Translated 400 sents for da-de
INFO: 2025-05-10 10:52:02 - [🕒]: Retrying da-el...
INFO: 2025-05-10 10:56:00 - [✔️]: Translated 400 sents for da-el
INFO: 2025-05-10 10:58:23 - [✔️]: Translated 400 sents for da-es
INFO: 2025-05-10 11:03:25 - [🕒]: Retrying da-fi...
INFO: 2025-05-10 11:08:50 - [✔️]: Translated 400 sents for da-fi
INFO: 2025-05-10 11:11:41 - [✔️]: Translated 400 sents for da-fr
INFO: 2025-05-10 11:13:27 - [✔️]: Translated 400 sents for da-it
INFO: 2025-05-10 11:15:24 - [✔️]: Translated 400 sents for da-nl
INFO: 2025-05-10 11:18:14 - [✔️]: Translated 400 sents for da-pt
INFO: 2025-05-10 11:20:03 - [✔️]: Translated 400 sents for da-sv
INFO: 2025-05-10 11:20:03 - [🏁]: Task took 2163.63s
INFO: 2025-05-10 11:22:12 - [🚀]: Selected task for flores_plus-de - gpt-

### Observations
* The errors still occured but only our own retries were triggered, reducing the translation time.
* The error message has changed, it was not a Gateway Timeout anymore but closed connection

In [26]:
!cat proc4.log | grep -P "(Retry|skipping)"

INFO: 2025-05-10 10:52:02 - [🕒]: Retrying da-el...
INFO: 2025-05-10 11:03:25 - [🕒]: Retrying da-fi...
INFO: 2025-05-10 11:34:33 - [🕒]: Retrying de-fi...
INFO: 2025-05-10 11:39:55 - [🕒]: Retrying de-fi...
INFO: 2025-05-10 11:45:26 - [⏭️]: Failed 2 times, skipping de-fi...
INFO: 2025-05-10 13:13:54 - [🕒]: Retrying fi-el...
INFO: 2025-05-10 13:45:27 - [🕒]: Retrying fr-de...


In [27]:
!cat proc4.log | grep -P "(]: Error)"

ERROR: 2025-05-10 10:52:02 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 11:03:25 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 11:34:33 - [🔥]: Error The server had an error while processing your request. Sorry about that!
ERROR: 2025-05-10 11:45:26 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 13:13:54 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 13:45:27 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)


* This gives us the following explanation on why these errors occured in the first place: 
    * The input prompt could be potentially the reason, however, that would not explain why it succeeded for most pairs and triggered errors only for some.
    * Most pairs that were retried (also in procedure 2 and 3, including OpenAI's retries) where of type: lang-el or lang-fi, although individual outliers exist.
    * We assume that GPT4.1 takes longer to produce output tokens when translating into Greek or Finnish. 
    * For Greek, it could be related to the usage of non-ASCII characters. For Finnish, due to the linguistic complexity of the language itself.
    * If it takes longer to process these two cases, it is more likely to for a Gateway Timeout to trigger as it exceeds a fixed time period defined by their servers. 
        * The Gateway Timeout 504 error message does not indicate such, but the new error message with "peer closed connection" does. 
    * The assumption is that both error types were triggered by the same root cause, even though procedure 2 & 3 used request-response and procedure 4 streaming.

In [28]:
!cat proc4.log | grep -P "(Selected task|Task took|skipping)"

INFO: 2025-05-10 10:43:59 - [🚀]: Selected task for flores_plus-da - gpt-4.1-2025-04-14
INFO: 2025-05-10 11:20:03 - [🏁]: Task took 2163.63s
INFO: 2025-05-10 11:22:12 - [🚀]: Selected task for flores_plus-de - gpt-4.1-2025-04-14
INFO: 2025-05-10 11:45:26 - [⏭️]: Failed 2 times, skipping de-fi...
INFO: 2025-05-10 12:00:44 - [🏁]: Task took 2311.90s
INFO: 2025-05-10 12:01:40 - [🚀]: Selected task for flores_plus-el - gpt-4.1-2025-04-14
INFO: 2025-05-10 12:32:12 - [🏁]: Task took 1832.04s
INFO: 2025-05-10 12:32:58 - [🚀]: Selected task for flores_plus-es - gpt-4.1-2025-04-14
INFO: 2025-05-10 13:01:12 - [🏁]: Task took 1693.26s
INFO: 2025-05-10 13:02:43 - [🚀]: Selected task for flores_plus-fi - gpt-4.1-2025-04-14
INFO: 2025-05-10 13:37:24 - [🏁]: Task took 2081.82s
INFO: 2025-05-10 13:38:14 - [🚀]: Selected task for flores_plus-fr - gpt-4.1-2025-04-14
INFO: 2025-05-10 14:12:26 - [🏁]: Task took 2051.53s
INFO: 2025-05-10 14:15:36 - [🚀]: Selected task for flores_plus-it - gpt-4.1-2025-04-14
INFO: 2025-

* We also noticed that on average, tasks in procedure 4 were faster than the ones in procedure 3. 
* It could be due to streaming or the reduced the number of retries (since streaming did only employ our retries, not OpenAI's)

## Procedure 5
* In hindsight, Procedure 5 might have been not necessary as it is was possible to retrieve some of the pairs that the previous API calls missed using the logs on OpenAI's platform, since a response was still generated and could be acessed with the API.
* However, at the time of procedure 5, this was not known and we just decided to try all failed pairs again, using the streaming approach of procedure 5 and setting our own max retries to 5.
* Thus, two tasks were run, to translate the pairs: fi-el, it-el, it-fi, sv-el from the EuroParl dataset and de-fi from the FLORES+ dataset.
* The following two CLI commands were ran:
```sh
python proc5.py run -m gpt-4.1-2025-04-14 -d europarl
python proc5.py run -m gpt-4.1-2025-04-14 -d flores_plus
```
* 4/5 pairs were translated, the pair it-el was skipped even after 5 retries. 
* We still obtained it through the method applied in [`it-el.ipynb`](https://github.com/na50r/110_bleu/blob/main/it-el.ipynb) but it can be considered the only pair that we never fully obtained through an API call directly, even with streaming.

In [29]:
proc5 = join(task_folder, 'proc5')
proc5_ep = join(proc5, 'europarl')
proc5_flores = join(proc5, 'flores_plus')
tl = 'gpt-4.1-2025-04-14'
proc5_flores_file = join(proc5_flores, join(tl, 'task.json'))
proc5_ep_file = join(proc5_ep, join(tl, 'task.json'))
proc5_task_ids = []
for fp in [proc5_ep_file, proc5_flores_file]:
    with open(fp) as f:
        task = json.load(f)
    proc5_task_ids.append(task['task_id'])

with open(join(task_folder, 'proc5.jsonl')) as f:
    logs = [json.loads(ln) for ln in f]

total = 0
proc5_task_ids2_logs = {i:[] for i in proc5_task_ids}
for task_id in proc5_task_ids:
    for log in logs:
        if log['id'].startswith(task_id):
            total+=1
            proc5_task_ids2_logs[task_id].append(log)

for task_id, logs in proc5_task_ids2_logs.items():
    print(task_id, len(logs))

49239a46-0014-4369-bcf7-c9f77283d3c9 3
b047d921-dd0a-4185-b000-c4ca55051693 1


In [30]:
for task in [t for t in os.listdir(proc5)]:
    files = os.listdir(join(proc5, join(task, 'gpt-4.1-2025-04-14')))
    print(join(proc5, join(task, 'gpt-4.1-2025-04-14')), len(files))

tasks\proc5\europarl\gpt-4.1-2025-04-14 4
tasks\proc5\flores_plus\gpt-4.1-2025-04-14 2


In [31]:
print(f'Proc5 {total}/5')

Proc5 4/5


In [32]:
!cat proc5.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-10 19:38:28 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-10 19:38:28 - [🏁]: Starting task 49239a46-0014-4369-bcf7-c9f77283d3c9 on commit 167ca14
INFO: 2025-05-10 19:42:52 - [✔️]: Translated 400 sents for fi-el
INFO: 2025-05-10 19:47:53 - [🕒]: Retrying it-el...
INFO: 2025-05-10 19:53:25 - [🕒]: Retrying it-el...
INFO: 2025-05-10 19:58:57 - [🕒]: Retrying it-el...
INFO: 2025-05-10 20:04:29 - [🕒]: Retrying it-el...
INFO: 2025-05-10 20:10:00 - [🕒]: Retrying it-el...
INFO: 2025-05-10 20:15:32 - [⏭️]: Failed 5 times, skipping it-el...
INFO: 2025-05-10 20:20:18 - [✔️]: Translated 399 sents for it-fi
INFO: 2025-05-10 20:24:46 - [✔️]: Translated 400 sents for sv-el
INFO: 2025-05-10 20:24:46 - [🏁]: Task took 2778.48s
INFO: 2025-05-10 20:26:22 - [🚀]: Selected task for flores_plus - gpt-4.1-2025-04-14
INFO: 2025-05-10 20:26:22 - [🏁]: Starting task b047d921-dd0a-4185-b000-c4ca55051693 on commit 167ca14
INFO: 2025-05-10 20:29:51 - [✔️]: Translated 400 sents for de

### Observation
* There is not much to add here, we only know that it really struggles with Italian to Greek for whatever reason.

In [33]:
!cat proc5.log | grep -P "(]: Error)"

ERROR: 2025-05-10 19:47:53 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 19:53:25 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 19:58:57 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 20:04:29 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 20:10:00 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)
ERROR: 2025-05-10 20:15:32 - [🔥]: Error peer closed connection without sending complete message body (incomplete chunked read)


In [34]:
!cat proc5.log | grep -P "(Selected task|Task took|skipping)"

INFO: 2025-05-10 19:38:28 - [🚀]: Selected task for europarl - gpt-4.1-2025-04-14
INFO: 2025-05-10 20:15:32 - [⏭️]: Failed 5 times, skipping it-el...
INFO: 2025-05-10 20:24:46 - [🏁]: Task took 2778.48s
INFO: 2025-05-10 20:26:22 - [🚀]: Selected task for flores_plus - gpt-4.1-2025-04-14
INFO: 2025-05-10 20:29:51 - [🏁]: Task took 209.09s


## Procedure 6
* In procedure 6, we translated the remaining 90 pairs from the EuroParl and Flores+ dataset using DeepL. We didn't do any fancy batching as we knew it should work without issues and it did.
* Thus, only two tasks were run with the following CLI commands:
```sh
python proc6.py run -m deepl_document -d europarl
python proc6.py run -m deepl_document -d flores_plus
```
* Everything was translated, no issues.

In [35]:
proc6 = join(task_folder, 'proc6')
proc6_ep = join(proc6, 'europarl')
proc6_flores = join(proc6, 'flores_plus')
tl = 'deepl_document'
proc6_task_ids = []
proc6_flores_file = join(proc6_flores, join(tl, 'task.json'))
proc6_ep_file = join(proc6_ep, join(tl, 'task.json'))
for fp in [proc6_ep_file, proc6_flores_file]:
    with open(fp) as f:
        task = json.load(f)
    proc6_task_ids.append(task['task_id'])

with open(join(task_folder, 'proc6.jsonl')) as f:
    logs = [json.loads(ln) for ln in f]

total = 0
proc6_task_ids2_logs = {i:[] for i in proc6_task_ids}
for task_id in proc6_task_ids:
    for log in logs:
        if log['id'].startswith(task_id):
            total+=1
            proc6_task_ids2_logs[task_id].append(log)

for task_id, logs in proc6_task_ids2_logs.items():
    print(task_id, len(logs))

fbf0130b-df94-4fcf-82b1-9635bb8ebe1e 90
dbe3476f-fc3c-40b9-9b1d-dae0b55a9d0f 90


In [36]:
for task in [t for t in os.listdir(proc6)]:
    files = os.listdir(join(proc6, join(task, 'deepl_document')))
    print(join(proc6, join(task, 'deepl_document')), len(files))

tasks\proc6\europarl\deepl_document 91
tasks\proc6\flores_plus\deepl_document 91


In [37]:
print(f'Proc6 {total}/180')

Proc6 180/180


In [38]:
!cat proc6.log | grep -P "^(INFO: \d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d) - (\[|Retry)"

INFO: 2025-05-10 20:42:10 - [🚀]: Selected task for europarl - deepl_document
INFO: 2025-05-10 20:42:10 - [🏁]: Starting task fbf0130b-df94-4fcf-82b1-9635bb8ebe1e on commit 601fdda
INFO: 2025-05-10 20:42:27 - [✔️]: Translated 400 sents for da-fr
INFO: 2025-05-10 20:42:39 - [✔️]: Translated 400 sents for da-de
INFO: 2025-05-10 20:42:50 - [✔️]: Translated 400 sents for da-it
INFO: 2025-05-10 20:43:01 - [✔️]: Translated 400 sents for da-sv
INFO: 2025-05-10 20:43:18 - [✔️]: Translated 400 sents for da-pt
INFO: 2025-05-10 20:43:34 - [✔️]: Translated 400 sents for da-fi
INFO: 2025-05-10 20:43:51 - [✔️]: Translated 400 sents for da-nl
INFO: 2025-05-10 20:44:08 - [✔️]: Translated 400 sents for da-el
INFO: 2025-05-10 20:44:25 - [✔️]: Translated 400 sents for da-es
INFO: 2025-05-10 20:44:31 - [✔️]: Translated 400 sents for de-it
INFO: 2025-05-10 20:44:42 - [✔️]: Translated 400 sents for de-sv
INFO: 2025-05-10 20:44:49 - [✔️]: Translated 400 sents for de-pt
INFO: 2025-05-10 20:45:00 - [✔️]: Transla

### Observation
* Not much to say except that DeepL is indeed much faster than GPT4.1

In [39]:
!cat proc6.log | grep -P "(Selected task|Task took|skipping)"

INFO: 2025-05-10 20:42:10 - [🚀]: Selected task for europarl - deepl_document
INFO: 2025-05-10 21:02:52 - [🏁]: Task took 1242.66s
INFO: 2025-05-10 21:03:20 - [🚀]: Selected task for flores_plus - deepl_document
INFO: 2025-05-10 21:21:13 - [🏁]: Task took 1072.50s


* To put this into perspective, it took DeepL less time to translate 90 pairs than it took GPT4.1 to translate 9.

## Final Remarks

* You can use the Commit hash to infer the procedures, there are only 6 distinct commit hashes

In [40]:
!cat proc*.log | grep -Po "(?<=on commit )(.+)$" | uniq

66ef922
f2cfbfa
61954df
7124518
167ca14
601fdda


* This notebook acts as a way to provide logs to the public. The `.log` files themselves cannot be published due to error messages including private informationt
* This notebook provides documentation and transparancy to all 6 procedures ran to obtain 40 + 220 + 219 translations obtained for 110 pairs, using datasets OPUS100, FLORES+ and EuroParl with DeepL and GPT4.1. 
    * The remaining translation for it-el was obtained using the method in [`it-el.ipynb`](https://github.com/na50r/110_bleu/blob/main/it-el.ipynb)